# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

# Imports

In [69]:
# imports

import os
from typing import List
from dotenv import load_dotenv
from IPython.display import  display, Audio
from openai import OpenAI
from io import BytesIO
import gradio as gr

# OpenAI API key

In [70]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openai = OpenAI()

model_options = {"llama3.2": ollama,"gpt-4o-mini" : openai }

OpenAI API Key exists and begins sk-proj-


In [ ]:
# Might need to run ollama serve
!ollama pull llama3.2

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


# SYSTEM prompt

In [72]:
# set up environment

def system_prompt():
    return f"""
    You are a tech expert and know every coding language, and can give 
    nice, detailed and simple explanations for the given questions.  
    
    """

In [73]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    display(Audio(output_filename, autoplay=True))

In [74]:
def listener(audio_file):
    with open(audio_file, "rb") as audio:
        transcript = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio
        )
    return transcript.text

In [75]:
def chat(cleared_entry, history,model):
    messages = [{"role": "system", "content": system_prompt()}] + history

    print(messages)
    print(model_options)
    stream = model_options[model.lower()].chat.completions.create(model=model.lower(), messages=messages, stream=True)
    response = ""
    
    for chunk in stream:
        try:
            response += chunk.choices[0].delta.content or ''
            yield history + [{"role": "assistant", "content": response}]  
        except Exception as e:
            print(f"Streaming error: {e}")
            yield "Sorry, there was an error processing your request."
    talker(response)


In [76]:
with gr.Blocks() as ui:

    gr.Markdown("## AI Chat Assistant")
    gr.Markdown("**Select your preferred AI model:**")
    
    model_dropdown = gr.Dropdown(
        choices=["GPT-4o-mini", "llama3.2"], 
        value="GPT-4o-mini",  # default selection
        label="Choose Model"
    )

    
    with gr.Row():
        chatbot = gr.Chatbot(height=200, type="messages")
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        # Audio input for voice messages
        audio_input = gr.Audio(
            sources=["microphone", "upload"], 
            type="filepath", 
            label="🎙️ Voice Message"
        )
    with gr.Row():
        voice_submit = gr.Button("Send Voice Message", variant="secondary")
        clear = gr.Button("Clear")


    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    def process_voice_input(audio_file):
        """Convert voice to text and put it in the text box"""
        if audio_file is not None:
            transcribed_text = listener(audio_file)
            if transcribed_text and not transcribed_text.startswith("Error"):
                return transcribed_text
        return ""

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=[entry,chatbot,model_dropdown], outputs=[chatbot]
    )

    voice_submit.click(
        process_voice_input,
        inputs=[audio_input],
        outputs=[entry]
    )
    
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': '\n    You are a tech expert and know every coding language, and can give \n    nice, detailed and simple explanations for the given questions.  \n\n    '}, {'role': 'user', 'metadata': None, 'content': 'hello ther', 'options': None}]
{'llama3.2': <openai.OpenAI object at 0x7f748420f5c0>, 'gpt-4o-mini': <openai.OpenAI object at 0x7f748420dd60>}


[{'role': 'system', 'content': '\n    You are a tech expert and know every coding language, and can give \n    nice, detailed and simple explanations for the given questions.  \n\n    '}, {'role': 'user', 'metadata': None, 'content': 'hello ther', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today? If you have any questions about coding, programming languages, or technology in general, feel free to ask!', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'What a nice day', 'options': None}]
{'llama3.2': <openai.OpenAI object at 0x7f748420f5c0>, 'gpt-4o-mini': <openai.OpenAI object at 0x7f748420dd60>}
